<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=bb2dedcadfc1c3147554e217306c3082660f3f1599e7eb819158dd918494d449
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-09 15:18:25
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 75.35 K (0.54%)
Current PnL: -20.40 L (-13.59%)
CY Booked + Current PnL: -8.96 L (-5.97%)
-------------------
Total profit:  1.74 L
Total loss:  -22.14 L
-------------------
Total Booked + Current PnL: 18.02 L (14.63%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.1%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.91 L (62.27%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 8.25%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.66,121799.0,21295.0,7807.0,2.77,21.19,6.41,28.96,79.0,2.73,0.87,62.35,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.81,40.0,M-SC,1.00,83486.0,-17291.0,17382.0,0.31,-17.16,20.82,0.09,245.0,-0.99,0.59,9.68,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.63,48.0,X-MC,3.06,189300.0,24630.0,18703.0,0.03,14.96,9.88,26.31,89.0,1.32,1.35,24.13,X40N,NTT,AC
19,CIPLA,1662.74,-22.18,45.0,X-LC,3.67,207322.0,2822.0,20483.0,1.25,1.38,9.88,11.39,10.0,0.14,1.47,10.46,X40N,ATH,PHARMA
84,VOLTAS,1530.00,-1.22,53.0,X-MC,1.17,207315.0,15573.0,22183.0,1.08,8.12,10.70,19.69,99.0,0.70,1.47,15.47,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SONACOMS,1006.00,-39.60,53.0,M-SC,13.66,75232.0,-25927.0,100818.0,6.08,-25.63,134.01,74.03,202.0,-0.26,0.54,6.09,AR,BTT,AUTO
55,RAJOOENG,143.10,-39.32,48.0,H-SC,16.32,92000.0,-10500.0,51097.0,5.87,-10.24,55.54,39.61,114.0,-0.21,0.65,12.36,AR,ATH,MISC
21,COFFEEDAY,80.00,-43.66,51.0,L-SC,29.49,81984.0,-31565.0,71613.0,4.99,-27.80,87.35,35.27,268.0,-0.44,0.58,100.66,XR,NTT,HOTELS
32,HINDZINC,730.22,36.83,75.0,M-LC,7.40,223133.0,18057.0,94519.0,4.58,8.80,42.36,54.89,52.0,0.19,1.59,35.02,X5K,ATH,METALS
60,SAIL,228.00,49.16,61.0,M-MC,12.66,234881.0,9919.0,157746.0,3.55,4.41,67.16,74.53,192.0,0.06,1.67,38.08,XY24,BTT,STEEL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.00,3721.62,54.0,M-SC,6.25,171513.0,-25619.0,115445.0,-2.95,-13.00,67.31,45.57,236.0,-0.22,1.22,21.60,XY24,NTT,AUTO
18,CERA,9475.00,-24.11,35.0,H-SC,1.69,138414.0,-37489.0,79505.0,-2.57,-21.31,57.44,23.89,149.0,-0.47,0.98,19.54,OX40N,NTT,CERAMICS
12,BATAINDIA,2096.00,-35.01,43.0,X-SC,4.41,97259.0,-31411.0,76708.0,-2.43,-24.41,78.87,35.20,219.0,-0.41,0.69,11.63,X40,NTT,FOOTWEAR
63,SFL,1287.00,28.52,36.0,M-SC,6.91,169790.0,-92428.0,162251.0,-1.70,-35.25,95.56,26.63,229.0,-0.57,1.21,5.57,XY24,NTT,MISC
37,INDIAMART,4810.62,-54.45,43.0,H-SC,8.94,127306.0,3970.0,127662.0,-1.46,3.22,100.28,106.72,119.0,0.03,0.91,27.35,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,61.0,H-SC,13.52,129870.0,-12609.0,32130.0,0.84,-8.85,24.74,13.70,163.0,-0.39,0.92,53.28,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.81,40.0,M-SC,1.00,83486.0,-17291.0,17382.0,0.31,-17.16,20.82,0.09,245.0,-0.99,0.59,9.68,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,48.0,H-SC,1.97,220932.0,-48735.0,85059.0,0.36,-18.07,38.50,13.47,138.0,-0.57,1.57,12.34,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-21.69,55.0,H-MC,6.49,105896.0,-25939.0,68165.0,2.08,-19.68,64.37,32.03,98.0,-0.38,0.75,18.61,OX40N,NTT,IT
18,CERA,9475.0,-24.11,35.0,H-SC,1.69,138414.0,-37489.0,79505.0,-2.57,-21.31,57.44,23.89,149.0,-0.47,0.98,19.54,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.66,121799.0,21295.0,7807.0,2.77,21.19,6.41,28.96,79.0,2.73,0.87,62.35,MH,ATH,METALS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.16,61.0,M-MC,12.66,234881.0,9919.0,157746.0,3.55,4.41,67.16,74.53,192.0,0.06,1.67,38.08,XY24,BTT,STEEL
32,HINDZINC,730.22,36.83,75.0,M-LC,7.40,223133.0,18057.0,94519.0,4.58,8.80,42.36,54.89,52.0,0.19,1.59,35.02,X5K,ATH,METALS
86,WIPRO,420.00,-12.89,52.0,M-LC,5.65,152966.0,2021.0,107428.0,1.00,1.34,70.23,72.51,53.0,0.02,1.09,7.44,XR,NTT,IT
79,UNIONBANK,163.00,-13.18,54.0,M-LC,8.96,156656.0,15816.0,28668.0,1.29,11.23,18.30,31.59,66.0,0.55,1.11,40.69,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,66.0,H-MC,11.51,193450.0,13642.0,98776.0,0.76,7.59,51.06,62.52,88.0,0.14,1.38,44.26,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.66,121799.0,21295.0,7807.0,2.77,21.19,6.41,28.96,79.0,2.73,0.87,62.35,MH,ATH,METALS
32,HINDZINC,730.22,36.83,75.0,M-LC,7.40,223133.0,18057.0,94519.0,4.58,8.80,42.36,54.89,52.0,0.19,1.59,35.02,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,66.0,H-MC,11.51,193450.0,13642.0,98776.0,0.76,7.59,51.06,62.52,88.0,0.14,1.38,44.26,XR,NTT,BANKS
37,INDIAMART,4810.62,-54.45,43.0,H-SC,8.94,127306.0,3970.0,127662.0,-1.46,3.22,100.28,106.72,119.0,0.03,0.91,27.35,AR,ATH,MISC
86,WIPRO,420.00,-12.89,52.0,M-LC,5.65,152966.0,2021.0,107428.0,1.00,1.34,70.23,72.51,53.0,0.02,1.09,7.44,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,26.0,X-SC,37.82,52100.0,-12906.0,163834.0,-0.75,-19.85,314.46,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
23,DABUR,735.00,-13.02,31.0,X-MC,3.91,235468.0,-12600.0,121007.0,1.11,-5.08,51.39,43.70,102.0,-0.10,1.67,8.63,XY24,BTT,FMCG
5,ASIANPAINT,3465.66,-18.55,32.0,X-LC,5.63,205436.0,-46332.0,99534.0,0.26,-18.40,48.45,21.13,27.0,-0.47,1.46,10.50,X40,ATH,PAINTS
29,HAVELLS,2069.16,-6.46,34.0,X-MC,0.56,231910.0,-15821.0,90886.0,-1.12,-6.39,39.19,30.30,92.0,-0.17,1.65,6.06,X40,ATH,ELECTRICAL
22,COLPAL,3767.14,-5.42,35.0,X-MC,7.08,220940.0,-42425.0,155785.0,-0.37,-16.11,70.51,43.04,84.0,-0.27,1.57,2.57,XY25,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-6.46,34.0,X-MC,0.56,231910.0,-15821.0,90886.0,-1.12,-6.39,39.19,30.30,92.0,-0.17,1.65,6.06,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,-1.22,53.0,X-MC,1.17,207315.0,15573.0,22183.0,1.08,8.12,10.70,19.69,99.0,0.70,1.47,15.47,XY25,NTT,AC
34,ICICIGI,2260.25,-19.18,47.0,X-MC,1.82,139164.0,3031.0,28097.0,0.13,2.23,20.19,22.86,91.0,0.11,0.99,15.12,X40,ATH,INSURANCE
73,TATAMOTORS,1065.00,-21.50,45.0,X-LC,1.90,248201.0,-28322.0,138397.0,0.32,-10.24,55.76,39.81,54.0,-0.20,1.77,18.95,XY24,NTT,AUTO
65,SIEMENS,4671.50,-1.11,49.0,H-LC,1.94,158830.0,-27265.0,74745.0,-1.25,-14.65,47.06,25.51,15.0,-0.36,1.13,17.05,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,26.0,X-SC,37.82,52100.0,-12906.0,163834.0,-0.75,-19.85,314.46,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
83,VBL,671.64,-21.10,40.0,X-LC,10.29,284261.0,-31581.0,144234.0,2.46,-10.00,50.74,35.67,5.0,-0.22,2.02,2.47,X40N,ATH,FMCG
22,COLPAL,3767.14,-5.42,35.0,X-MC,7.08,220940.0,-42425.0,155785.0,-0.37,-16.11,70.51,43.04,84.0,-0.27,1.57,2.57,XY25,ATH,FMCG
42,ITC,452.00,-39.61,41.0,X-LC,2.27,195576.0,-4562.0,25444.0,0.05,-2.28,13.01,10.44,4.0,-0.18,1.39,3.16,X40,NTT,FMCG
9,BAJAJHFL,181.50,-14.27,39.0,X-MC,4.87,180630.0,-19876.0,117030.0,-0.37,-9.91,64.79,48.45,90.0,-0.17,1.28,4.16,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,26.0,X-SC,37.82,52100.0,-12906.0,163834.0,-0.75,-19.85,314.46,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
56,RELAXO,1176.00,-39.59,39.0,X-SC,2.17,82353.0,-62807.0,138732.0,-1.04,-43.27,168.46,52.31,136.0,-0.45,0.59,10.23,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.01,43.0,X-SC,4.41,97259.0,-31411.0,76708.0,-2.43,-24.41,78.87,35.20,219.0,-0.41,0.69,11.63,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-5.61,48.0,X-SC,1.64,107296.0,5292.0,38637.0,1.37,5.19,36.01,43.06,122.0,0.14,0.76,23.19,X40N,ATH,MISC
33,HONAUT,58357.33,-22.88,46.0,X-SC,2.15,108720.0,-19218.0,66352.0,0.18,-15.02,61.03,36.84,143.0,-0.29,0.77,11.30,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-27.72,55.0,X-LC,12.13,287931.0,-58025.0,126258.0,1.19,-16.77,43.85,19.73,1.0,-0.46,2.05,6.05,X40,ATH,IT
40,INFY,2275.00,-17.44,56.0,X-LC,7.90,321310.0,8422.0,163258.0,0.90,2.69,50.81,54.87,3.0,0.05,2.29,9.52,X40,BTT,IT
42,ITC,452.00,-39.61,41.0,X-LC,2.27,195576.0,-4562.0,25444.0,0.05,-2.28,13.01,10.44,4.0,-0.18,1.39,3.16,X40,NTT,FMCG
83,VBL,671.64,-21.10,40.0,X-LC,10.29,284261.0,-31581.0,144234.0,2.46,-10.00,50.74,35.67,5.0,-0.22,2.02,2.47,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,42.0,H-LC,7.52,247128.0,-14491.0,133696.0,0.39,-5.54,54.10,45.57,7.0,-0.11,1.76,5.16,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,51.0,L-SC,29.49,81984.0,-31565.0,71613.0,4.99,-27.80,87.35,35.27,268.0,-0.44,0.58,100.66,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,40.0,H-SC,10.61,90330.0,-7650.0,29249.0,-0.95,-7.81,32.38,22.05,152.0,-0.26,0.64,31.30,XR,ATH,FINANCE
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.66,121799.0,21295.0,7807.0,2.77,21.19,6.41,28.96,79.0,2.73,0.87,62.35,MH,ATH,METALS
78,UJJIVANSFB,60.00,105.37,61.0,H-SC,13.52,129870.0,-12609.0,32130.0,0.84,-8.85,24.74,13.70,163.0,-0.39,0.92,53.28,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,64.0,M-SC,14.38,144918.0,-4302.0,148483.0,1.22,-2.88,102.46,96.62,208.0,-0.03,1.03,56.83,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,36.83,75.0,M-LC,7.40,223133.0,18057.0,94519.0,4.58,8.80,42.36,54.89,52.0,0.19,1.59,35.02,X5K,ATH,METALS
61,SAMMAANCAP,326.00,-184.09,64.0,M-SC,14.38,144918.0,-4302.0,148483.0,1.22,-2.88,102.46,96.62,208.0,-0.03,1.03,56.83,XY25,NTT,FINANCE
69,SURYODAY,240.00,64.47,64.0,H-SC,8.13,151756.0,-27315.0,93042.0,2.89,-15.25,61.31,36.71,135.0,-0.29,1.08,50.57,XR,NTT,BANKS
26,GREENPANEL,537.00,234.63,60.0,M-SC,4.91,149377.0,-33701.0,112675.0,-0.37,-18.41,75.43,43.14,230.0,-0.30,1.06,40.51,XY24,NTT,MISC
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.66,121799.0,21295.0,7807.0,2.77,21.19,6.41,28.96,79.0,2.73,0.87,62.35,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.20
1,25,43.56
2,50,75.21


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.89
LC    32.35
MC    23.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.95
X40      14.65
X40N     11.20
XR       10.95
XY25     10.37
AR        8.18
OX40N     7.63
X200      1.74
MH        1.63
X5K       1.59
SR        1.11
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.46
X-LC    22.92
X-MC    17.14
M-SC    12.50
H-MC     4.65
H-LC     4.63
X-SC     4.50
M-LC     3.79
M-MC     1.67
L-SC     1.43
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.75,-6.17,41.17
IT,12.91,-15.89,78.65
FINANCE,10.46,-14.55,69.87
MISC,6.91,-18.31,83.87
BANKS,6.35,-11.80,70.61
PAINTS,5.49,-19.56,37.38
ELECTRICAL,5.31,-10.29,48.98
INSURANCE,3.79,-3.64,42.38
AUTO,3.53,-16.14,71.66


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3220692.0,22
XR,1371024.0,14
AR,1181492.0,9
X40,781308.0,10
X40N,659762.0,9
OX40N,570735.0,10
XY25,479473.0,7
SR,265492.0,2
X5K,94519.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3421338.0,25
M-SC,1282641.0,15
X-MC,1123525.0,12
X-LC,1020555.0,13
X-SC,547092.0,6
H-MC,356664.0,4
H-LC,288946.0,3
L-SC,259021.0,3
M-LC,230615.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1202642.0      6
           AR         806761.0      5
           XR         801675.0      7
M-SC       XY24       688252.0      6
X-MC       XY24       590877.0      4
X-LC       X40        485653.0      5
X-SC       X40N       306550.0      4
H-SC       OX40N      266713.0      4
           SR         265492.0      2
X-MC       X40        218947.0      4
X-LC       X40N       217479.0      3
H-LC       AR         208441.0      2
X-LC       XY24       206757.0      2
H-MC       XY24       181916.0      1
X-MC       XY25       177968.0      2
L-SC       XR         169891.0      2
M-SC       AR         166290.0      2
X-SC       XY24       163834.0      1
M-MC       XY24       157746.0      1
M-SC       XY25       148483.0      1
           OX40N      146727.0      4
X-MC       X40N       135733.0      2
M-SC       XR         132889.0      2
X-LC       XY25       110666.0      3
M-LC       XR         107428.0      1
H-MC       XR          98776.0      1
M-LC       X5K         94519.0      1
L-SC       OX40N       89130.0      1
H-LC       X200        80505.0      1
H-SC       MH          78055.0      1
X-SC       X40         76708.0      1
H-MC       OX40N       68165.0      1
L-MC       XR          60365.0      1
L-LC       XY25        42356.0      1
M-LC       XY24        28668.0      1
H-MC       MH           7807.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
